In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.1.0'

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
def create_weight(shape):return tf.Variable(tf.truncated_normal(shape=shape,stddev=0.1,name='w'))
def create_bias(shape):return tf.Variable(tf.constant(0.1,shape=shape,name='b'))

In [5]:
def conv_layer(input,shape,name='conv'):
    w = create_weight(shape)
    tf.summary.histogram('weights',w)
    return tf.nn.conv2d(input,w,strides=[1,1,1,1],padding='SAME',name=name)

In [6]:
def max_pool(input):
    return tf.nn.max_pool(input,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [7]:
def fc(input,shape,name='fc'):
    w = create_weight(shape)
    b = create_bias([shape[1]])
    tf.summary.histogram('weights',w)
    tf.summary.histogram('bias',b)
    return (tf.matmul(input , w) + b)

In [8]:
def relu(input):
    return tf.nn.relu(input)

In [9]:
sess = tf.InteractiveSession()

In [10]:
x = tf.placeholder(dtype=tf.float32,shape=[None,784],name='x')
y = tf.placeholder(dtype=tf.float32,shape=[None,10],name='y')
x_image = tf.reshape(x,shape=[-1,28,28,1])
tf.summary.image('image',x_image,max_outputs=3)
conv1 = max_pool(conv_layer(x_image,shape=[5,5,1,32],name='conv1'))
conv2 = max_pool(conv_layer(conv1,shape=[5,5,32,64],name='conv2'))
flattened = tf.reshape(conv2,shape=[-1,7*7*64])
fc1 = relu(fc(flattened,shape=[7*7*64,1024],name='fc1'))
fc2 = fc(fc1,shape=[1024,10],name='fc2')

In [11]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=fc2),name='loss')
tf.summary.scalar('loss',cross_entropy)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [12]:
correct_prediction = tf.equal(tf.argmax(fc2,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,dtype=tf.float32))
tf.summary.scalar('accuracy',accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [13]:
summ = tf.summary.merge_all()

In [14]:
sess.run(tf.global_variables_initializer())

In [15]:
writer = tf.summary.FileWriter('demo1/')
writer.add_graph(sess.graph)

In [19]:
for i in range(1000):
    batch = mnist.train.next_batch(100)
    if i%5 == 0:
        [a,s] = sess.run([accuracy,summ],feed_dict={x:batch[0],y:batch[1]})
        writer.add_summary(s,i)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0],y:batch[1]})
        print(f'step {i} and accuracy is {train_accuracy}')
    train_step.run(feed_dict={x:batch[0],y:batch[1]})

step 0 and accuracy is 0.9700000286102295
step 100 and accuracy is 0.9600000381469727
step 200 and accuracy is 0.9700000286102295
step 300 and accuracy is 0.9800000190734863
step 400 and accuracy is 0.9800000190734863
step 500 and accuracy is 0.9700000286102295
step 600 and accuracy is 0.9900000095367432
step 700 and accuracy is 0.9900000095367432
step 800 and accuracy is 1.0
step 900 and accuracy is 1.0


In [ ]:
test_accuracy = accuracy.eval(feed_dict={x:mnist.test.images,y:mnist.test.labels})

In [ ]:
print(f'test accuracy is {test_accuracy}')